<a href="https://colab.research.google.com/github/noambassat/SupremeCourtClassifier/blob/main/5_second_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Embedding + first classifier

In [1]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoModel
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_validate, cross_val_score, StratifiedKFold,train_test_split,cross_val_predict
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score,make_scorer, accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from imblearn.under_sampling import RandomUnderSampler


In [2]:
data = pd.read_csv("full_final_df.csv")

In [3]:
data["outcome_category"].unique()

array([1, 0, 2])


## Training

In [ ]:

tokenizer = AutoTokenizer.from_pretrained("dean-ai/sentence_transformer_Legal-heBERT")
model = AutoModel.from_pretrained("dean-ai/sentence_transformer_Legal-heBERT")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/498M [00:00<?, ?B/s]

In [ ]:
# פונקציה שמקבלת טקסט ומחזירה את האימבדינג כוקטור
def get_embeddings(text):
    # המרת הטקסט לפורמט מתאים למודל
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)
    # חישוב האימבדינג של המודל
    outputs = model(**inputs)
    # חישוב ממוצע הוקטורים עבור כל טקסט
    embeddings = outputs.last_hidden_state.mean(dim=1).detach().numpy()
    return embeddings[0]

In [ ]:
# פונקציה לחיתוך הטקסט בהתאם לכללים
def trim_text(text):
    min_length = 500  # אורך מינימלי לשמירה
    last_sen = 30     # מספר תווים להסרה מסוף הטקסט

    # שמירת אורך מקורי לאבחון
    original_length = len(text)

    # הסרת מספר תווים מסוף הטקסט
    text = text[:-last_sen]

    # חיתוך ל-min_length האחרונים אם הטקסט ארוך יותר מהמינימום
    if len(text) > min_length:
        text = text[-min_length:]

    # הדפסת פידבק רק אם הטקסט עבר שינוי
    if len(text) != original_length:
        print(f"Trimmed Text (Original Length: {original_length}, Trimmed Length: {len(text)}):")
        formatted_text = "\n".join([text[i:i+80] for i in range(0, len(text), 80)])
        print(formatted_text)
        print("##*******************************************************####")
    return text


# הסרת שורות עם ערכים חסרים בעמודת הטקסט או בעמודת המטרה
data = data.dropna(subset=["גוף המסמך", "outcome_category"])

# חיתוך הטקסטים בעמודת "גוף המסמך"
data["גוף המסמך"] = data["גוף המסמך"].apply(trim_text)

# # שמירת התוצאה לקובץ
# file_path = 'final_trimmed_data.csv'
# data.to_csv(file_path, index=False, encoding='utf-8')

# print(f"הקובץ נשמר בהצלחה בנתיב: {file_path}")


In [ ]:
from sklearn.metrics import precision_recall_curve, accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, make_scorer
from sklearn.model_selection import cross_val_predict, cross_validate
import joblib


# עדכון עמודת המטרה ואימבדינג לאחר חיתוך הטקסטים
y = data["outcome_category"].values
embeddings = np.array([get_embeddings(text) for text in data["גוף המסמך"]])

In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_predict, cross_validate
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, make_scorer
from imblearn.under_sampling import RandomUnderSampler
import numpy as np
import joblib

# בדיקת הקטגוריות ותוויות היעד
print("Unique categories in 'y_numeric':", np.unique(data["outcome_category"]))

# חישוב התפלגות הקטגוריות המקורית
unique, counts = np.unique(data["outcome_category"], return_counts=True)
class_distribution = dict(zip(unique, counts))
print("Original class distribution:", class_distribution)

# הגדרת יחס הדגימה: undersampling למחלקה הגדולה ביותר
major_class = max(class_distribution, key=class_distribution.get)  # מחלקה עם מספר הדוגמאות הגבוה ביותר
sampling_strategy = {
    major_class: int(class_distribution[major_class] * 0.6),  # הפחתת מספר הדוגמאות ב-40%
    1: class_distribution[1],  # שמירה על כל הדוגמאות בקטגוריה "ערעור התקבל"
    2: class_distribution[2]   # שמירה על כל הדוגמאות בקטגוריה "אחר"
}
print("Sampling strategy:", sampling_strategy)

# ביצוע undersampling
undersampler = RandomUnderSampler(sampling_strategy=sampling_strategy, random_state=42)
X_resampled, y_resampled = undersampler.fit_resample(embeddings, data["outcome_category"])

# שמירת האינדקסים המקוריים
resampled_indices = undersampler.sample_indices_
joblib.dump(resampled_indices, 'resampled_indices.pkl')

# חישוב התפלגות לאחר undersampling
unique_resampled, counts_resampled = np.unique(y_resampled, return_counts=True)
resampled_distribution = dict(zip(unique_resampled, counts_resampled))
print("Resampled class distribution:", resampled_distribution)

# הגדרת Logistic Regression עם משקל מוגדל לקטגוריה 1
class_weights = {0: 1, 1: 3, 2: 1}  # משקל גבוה לקטגוריה 1
classifier = LogisticRegression(random_state=42, max_iter=1000, class_weight=class_weights, multi_class='multinomial')

# אימון המודל
classifier.fit(X_resampled, y_resampled)

# הגדרת מדדים לקרוס ולידציה
scoring = {
    'accuracy': make_scorer(accuracy_score),
    'precision_macro': make_scorer(precision_score, average='macro'),
    'recall_macro': make_scorer(recall_score, average='macro'),
    'f1_macro': make_scorer(f1_score, average='macro')
}

# ביצוע Cross Validation
cv_results = cross_validate(classifier, X_resampled, y_resampled, cv=5, scoring=scoring)

# הדפסת תוצאות קרוס ולידציה
print("\nCross-Validation Results:")
print("Accuracy:", cv_results['test_accuracy'])
print("Mean Accuracy:", cv_results['test_accuracy'].mean())
print("Precision (Macro):", cv_results['test_precision_macro'])
print("Mean Precision (Macro):", cv_results['test_precision_macro'].mean())
print("Recall (Macro):", cv_results['test_recall_macro'])
print("Mean Recall (Macro):", cv_results['test_recall_macro'].mean())
print("F1 Score (Macro):", cv_results['test_f1_macro'])
print("Mean F1 Score (Macro):", cv_results['test_f1_macro'].mean())

# חיזוי תוויות על כל קפל לצורך יצירת קונפיוז'ן מטריקס
y_pred = cross_val_predict(classifier, X_resampled, y_resampled, cv=5)

# חישוב קונפיוז'ן מטריקס
conf_matrix = confusion_matrix(y_resampled, y_pred)
print("\nConfusion Matrix:")
print(conf_matrix)

# הדפסת FN, FP
fn_count = np.sum((y_resampled == 1) & (y_pred != 1))
fp_count = np.sum((y_resampled != 1) & (y_pred == 1))
print(f"False Negatives (FN): {fn_count}")
print(f"False Positives (FP): {fp_count}")

# שמירת המודל
joblib.dump(classifier, 'classifier_multiclass_model_with_weights.pkl')
print("Model saved successfully!")


In [ ]:


# בדיקה ראשונית של הקטגוריות
print("Unique categories in 'y_numeric':", np.unique(data["outcome_category"]))

# חישוב התפלגות הקטגוריות המקורית
unique, counts = np.unique(data["outcome_category"], return_counts=True)
class_distribution = dict(zip(unique, counts))
print("Original class distribution:", class_distribution)

# ביצוע Undersampling למחלקה הדומיננטית בלבד
sampling_strategy = {
    0: min(class_distribution[0], int(class_distribution[0] * 0.6)),  # קטגוריה 0
    1: class_distribution[1],  # שימור המחלקה ללא שינוי
    2: class_distribution[2]   # שימור המחלקה ללא שינוי
}
print("Sampling strategy:", sampling_strategy)

undersampler = RandomUnderSampler(sampling_strategy=sampling_strategy, random_state=42)
X_resampled, y_resampled = undersampler.fit_resample(embeddings, data["outcome_category"])

# שמירת אינדקסים מקוריים
resampled_indices = undersampler.sample_indices_
joblib.dump(resampled_indices, 'resampled_indices.pkl')

# חישוב התפלגות לאחר Undersampling
unique_resampled, counts_resampled = np.unique(y_resampled, return_counts=True)
resampled_distribution = dict(zip(unique_resampled, counts_resampled))
print("Resampled class distribution:", resampled_distribution)

# הגדרת מסווג לוגיסטי עם משקלות מותאמים
class_weights = {0: 1, 1: 3, 2: 5}  # משקל גבוה לקטגוריות 1 ו-2
classifier = LogisticRegression(random_state=42, max_iter=1000, class_weight=class_weights)

# אימון המודל
classifier.fit(X_resampled, y_resampled)

# הגדרת מדדים לקרוס ולידציה
scoring = {
    'accuracy': make_scorer(accuracy_score),
    'precision_macro': make_scorer(precision_score, average='macro'),
    'recall_macro': make_scorer(recall_score, average='macro'),
    'f1_macro': make_scorer(f1_score, average='macro')
}

# ביצוע Cross Validation
cv_results = cross_validate(classifier, X_resampled, y_resampled, cv=5, scoring=scoring)

# הדפסת תוצאות קרוס ולידציה
print("\nCross-Validation Results:")
print("Accuracy:", cv_results['test_accuracy'])
print("Mean Accuracy:", cv_results['test_accuracy'].mean())
print("Precision (Macro):", cv_results['test_precision_macro'])
print("Mean Precision (Macro):", cv_results['test_precision_macro'].mean())
print("Recall (Macro):", cv_results['test_recall_macro'])
print("Mean Recall (Macro):", cv_results['test_recall_macro'].mean())
print("F1 Score (Macro):", cv_results['test_f1_macro'])
print("Mean F1 Score (Macro):", cv_results['test_f1_macro'].mean())

# חיזוי תוויות על כל קפל לצורך יצירת קונפיוז'ן מטריקס
y_pred = cross_val_predict(classifier, X_resampled, y_resampled, cv=5)

# חישוב קונפיוז'ן מטריקס
conf_matrix = confusion_matrix(y_resampled, y_pred)
print("\nConfusion Matrix:")
print(conf_matrix)

# חישוב FN ו-FP
fn_count = np.sum((y_resampled == 1) & (y_pred != 1))
fp_count = np.sum((y_resampled != 1) & (y_pred == 1))
print(f"False Negatives (FN): {fn_count}")
print(f"False Positives (FP): {fp_count}")

# שמירת המודל
joblib.dump(classifier, 'classifier_weighted_model.pkl')
print("Model with adjusted class weights saved successfully!")
